# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
import sqlite3

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to C:\Users\Captain
[nltk_data]     Nemo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Captain
[nltk_data]     Nemo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Captain Nemo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertDatabaseName', engine)

# Assign message column as inputs and category columns as targets
X = df.message.values
y = df.iloc[:,4:].values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Split train and test datasets:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

# Training the pipeline:
pipeline.fit(X_train, y_train)
predicted = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
# Printing the classification report using scikit-learn method:
from sklearn.metrics import classification_report

# input datasets for classification report function should be dataframes:
category_names = df.iloc[:,4:].columns
y_test_df = pd.DataFrame(y_test, columns=category_names)
predicted_df = pd.DataFrame(predicted, columns = category_names)

print(classification_report(y_test_df, predicted_df, target_names= category_names, zero_division=1))

                        precision    recall  f1-score   support

               related       0.66      0.94      0.78      1688
               request       0.39      0.07      0.11       908
                 offer       1.00      0.00      0.00         4
           aid_related       0.37      0.07      0.12       972
          medical_help       1.00      0.00      0.00       137
      medical_products       1.00      0.00      0.00        77
     search_and_rescue       1.00      0.00      0.00        44
              security       1.00      0.00      0.00        35
              military       1.00      0.00      0.00         7
           child_alone       1.00      1.00      1.00         0
                 water       0.00      0.00      0.00       183
                  food       0.33      0.01      0.03       384
               shelter       0.00      0.00      0.00       275
              clothing       1.00      0.00      0.00        21
                 money       1.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# perform grid search with the chosen parameters 
parameters = {
    'vect__ngram_range': ((1,1),(1,2)),
    'clf__estimator__leaf_size': [30,40,50],
    'clf__estimator__n_neighbors':[5,7,9]
}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)
predicted = cv.predict(X_test)
print("\nBest Parameters:", cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
predicted_df = pd.DataFrame(predicted, columns = category_names)
print(classification_report(y_test_df, predicted_df, target_names= category_names))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [7]:
# adding a new extractor to be used as an additional transformer in out pipeline:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [8]:
# define a new fucntion for comparing different ML algorythms:
from sklearn.metrics import classification_report
category_names = df.iloc[:,4:].columns
def ClassifierComp(classifier):
    pipeline = Pipeline([
        ('Features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(classifier))
    ])
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
    pipeline.fit(X_train, y_train)
    predicted = pipeline.predict(X_test)
    y_test_df = pd.DataFrame(y_test, columns=category_names)
    predicted_df = pd.DataFrame(predicted, columns = category_names)
    report = classification_report(y_test_df, predicted_df, 
                                   target_names= category_names,
                                   output_dict=True, 
                                   zero_division=1)
    return report['weighted avg']['f1-score']

In [22]:
# import libraries for multioutput classifiers based on scikit-learn documentation
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
# iterating through the list of classifiers to compare their f1-score:
Classifiers = [DecisionTreeClassifier(), ExtraTreeClassifier(), ExtraTreesClassifier(), 
               RandomForestClassifier(), KNeighborsClassifier()]

for classifier in Classifiers:
    print(str(classifier)[:-2], ClassifierComp(classifier))

# Comp_table = pd.DataFrame(scores, columns=['Classifiers', 'f1-score'])
# Comp_table.sort_values(by='f1-score', ascending=False)

DecisionTreeClassifier 0.34020879155417455
ExtraTreeClassifier 0.3300385672894629
ExtraTreesClassifier 0.2445681402965983
RandomForestClassifier 0.2508412544373674
KNeighborsClassifier 0.25198990247067654


In [12]:
# According to the last cell's resurlt, we chose DecisionTreeClassifier as our estimator:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
category_names = df.iloc[:,4:].columns
pipeline = Pipeline([
    ('Features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
pipeline.fit(X_train, y_train)
predicted = pipeline.predict(X_test)

In [13]:
category_names = df.iloc[:,4:].columns
y_test_df = pd.DataFrame(y_test, columns=category_names)
predicted_df = pd.DataFrame(predicted, columns = category_names)

print(classification_report(y_test_df, predicted_df, target_names= category_names, zero_division=1))

                        precision    recall  f1-score   support

               related       0.66      0.70      0.68      1688
               request       0.38      0.34      0.36       908
                 offer       1.00      0.00      0.00         4
           aid_related       0.43      0.42      0.42       972
          medical_help       0.05      0.04      0.04       137
      medical_products       0.04      0.04      0.04        77
     search_and_rescue       0.00      0.00      0.00        44
              security       0.03      0.03      0.03        35
              military       0.00      0.00      0.00         7
           child_alone       1.00      1.00      1.00         0
                 water       0.12      0.10      0.11       183
                  food       0.17      0.14      0.15       384
               shelter       0.11      0.11      0.11       275
              clothing       0.00      0.00      0.00        21
                 money       0.00      

In [14]:
# now we can perform another grid search on this new estimator to be sure we have the best parameters
parameters = {
    'Features__text_pipeline__vect__ngram_range': ((1,1),(1,2)),
    'Features__text_pipeline__tfidf__use_idf': (True, False),
    'Features__text_pipeline__vect__max_df': [0.5,1.0],
    'Features__text_pipeline__vect__max_features': [100, 500, 1000, None],
    'Features__text_pipeline__tfidf__smooth_idf': (True, False)    
}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)
predicted = cv.predict(X_test)
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'Features__text_pipeline__tfidf__smooth_idf': False, 'Features__text_pipeline__tfidf__use_idf': False, 'Features__text_pipeline__vect__max_df': 0.5, 'Features__text_pipeline__vect__max_features': None, 'Features__text_pipeline__vect__ngram_range': (1, 1)}


### 9. Export your model as a pickle file

In [16]:
import pickle
pkl_filename = "DisasterResponse_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(cv.best_estimator_, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.